In [1]:
#model_implementation.ipynb

import cv2
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
import time

# Load the model
def load_model(model_path):
    return joblib.load(model_path)

# Extract features from the segmented image
import cv2

def extract_features(segmented_image):
    # Resize the segmented image to 16x16 pixels
    resized_image = cv2.resize(segmented_image, (16, 1))
    # Flatten the resized image to create a 1D array of 256 features
    features = resized_image.flatten()
    return features


def predict_species(model, features):
    print(f"Extracted features: {features}",model )  # Debugging statement to show features
    features = np.array(features).reshape(1, -1)  # Ensure features are in the correct shape
    predicted_species = model.predict(features)
    print(f"Predicted species: {predicted_species[0]}")  # Debugging statement to show prediction
    return predicted_species[0]  # Return the first prediction

def calculate_specificity(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    if cm.shape == (2, 2):  # Binary classification
        tn, fp, fn, tp = cm.ravel()
        return tn / (tn + fp) if (tn + fp) != 0 else 0.0
    else:  # Multiclass classification
        specificities = []
        for i in range(cm.shape[0]):
            tn = cm.sum() - (cm[i, :].sum() + cm[:, i].sum() - cm[i, i])
            fp = cm[:, i].sum() - cm[i, i]
            specificities.append(tn / (tn + fp) if (tn + fp) != 0 else 0.0)
        return sum(specificities) / len(specificities) if specificities else 0.0

def calculate_metrics(model, X_test, y_test):
    start_time = time.time()  # Record start time
    y_pred = model.predict(X_test)
    end_time = time.time()  # Record end time
    elapsed_time = end_time - start_time  # Calculate elapsed time
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=1)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=1)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=1)
    specificity = calculate_specificity(y_test, y_pred)

    return accuracy, precision, recall, specificity, f1, elapsed_time

# Main script
if __name__ == "__main__":
    # Load models
    rf_model = load_model('rf_model.pkl')
    logistic_model = load_model('logistic_model.pkl')
    svm_model = load_model('svm_model.pkl')

    # Load the CSV files
    feature_train = pd.read_csv('FeatureTrain.csv')
    feature_test = pd.read_csv('FeatureTest.csv')

    # Separate features and labels
    X_train = feature_train.drop(columns='Class').values
    y_train = feature_train['Class'].values
    X_test = feature_test.drop(columns='Class').values
    y_test = feature_test['Class'].values

    # Scale the data
    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Ensure the shape of the test data is correct
    print(f"Shape of scaled test data: {X_test_scaled.shape}")  # Should be (68, 16)

    # Calculate and print metrics for Random Forest model
    rf_accuracy, rf_precision, rf_recall, rf_specificity, rf_f1, rf_elapsed_time = calculate_metrics(rf_model, X_test_scaled, y_test)
    print(f"Random Forest Metrics:\n Accuracy: {rf_accuracy}\n Precision: {rf_precision}\n Recall: {rf_recall}\n Specificity: {rf_specificity}\n F1 Score: {rf_f1}\n Elapsed Time: {rf_elapsed_time}")

    # Calculate and print metrics for Logistic Regression model
    logistic_accuracy, logistic_precision, logistic_recall, logistic_specificity, logistic_f1, logistic_elapsed_time = calculate_metrics(logistic_model, X_test_scaled, y_test)
    print(f"Logistic Regression Metrics:\n Accuracy: {logistic_accuracy}\n Precision: {logistic_precision}\n Recall: {logistic_recall}\n Specificity: {logistic_specificity}\n F1 Score: {logistic_f1}\n Elapsed Time: {logistic_elapsed_time}")

    # Calculate and print metrics for SVM model
    svm_accuracy, svm_precision, svm_recall, svm_specificity, svm_f1, svm_elapsed_time = calculate_metrics(svm_model, X_test_scaled, y_test)
    print(f"SVM Metrics:\n Accuracy: {svm_accuracy}\n Precision: {svm_precision}\n Recall: {svm_recall}\n Specificity: {svm_specificity}\n F1 Score: {svm_f1}\n Elapsed Time: {svm_elapsed_time}")

   

Shape of scaled test data: (68, 16)
Random Forest Metrics:
 Accuracy: 0.9558823529411765
 Precision: 0.9705882352941176
 Recall: 0.9558823529411765
 Specificity: 0.9984848484848484
 F1 Score: 0.9549019607843138
 Elapsed Time: 0.021991252899169922
Logistic Regression Metrics:
 Accuracy: 0.8235294117647058
 Precision: 0.8475490196078431
 Recall: 0.8235294117647058
 Specificity: 0.9938927738927739
 F1 Score: 0.8190826330532213
 Elapsed Time: 0.0
SVM Metrics:
 Accuracy: 0.9558823529411765
 Precision: 0.9693627450980391
 Recall: 0.9558823529411765
 Specificity: 0.9984770784770785
 F1 Score: 0.9544817927170868
 Elapsed Time: 0.005014657974243164
